# Baseline model for Elle Ebène

## Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np

Les données proviennent du dossier raw_data : 1 dossier type 3, 1 dossier type 4.

Option 1: utilisation de Matplotlib. Problème : impossible de mettre toutes les images à la même taille (ou on le fait dans le modèle?).  
Option 2 : on commence par utiliser PIL, on met tout à la bonne taille, puis on passe en numpy array.  

=> Option 2

On commence par récupérer et explorer les images

In [3]:
from  PIL import Image

On fixe la hauteur de l'image, on fera un padding sur la largeur.

In [4]:
from elle_ebene.simple_preprocessing import get_images

In [5]:
type3_imgs = get_images("Type 3")
type4_imgs = get_images("Type 4")

On a bien des hauteurs fixes.  
On convertit tout ça en numpy arrays et en rgb (3 canaux, on supprime brutalement le 4ème canal = alpha)

On a des rgba et des rgb, il faut convertir les formats.

In [8]:
from elle_ebene.simple_preprocessing import to_numpy_rgb

In [9]:
type3_imgs_rgb = to_numpy_rgb(type3_imgs)
type4_imgs_rgb = to_numpy_rgb(type4_imgs)

On veut des images carrés de resizing_dim*resizing_dim

In [11]:
from elle_ebene.simple_preprocessing import squared_imgs

In [12]:
type3_imgs_squared = squared_imgs(type3_imgs_rgb)
type4_imgs_squared = squared_imgs(type4_imgs_rgb)

## Creation of the dataset

In [15]:
type3_labels = np.full((len(type3_imgs_squared),),3)
type4_labels = np.full((len(type4_imgs_squared),),4)

In [17]:
from elle_ebene.simple_preprocessing import create_dataset

In [18]:
X, y = create_dataset([type3_imgs_squared, type4_imgs_squared], [type3_labels, type4_labels])

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Model

Normalisation des photos (simple)

In [23]:
X_train = X_train / 255
X_test = X_test / 255

In [24]:
from tensorflow.keras.utils import to_categorical

In [25]:
y_train = to_categorical(y_train,5)[:,-2:]
y_test = to_categorical(y_test,5)[:,-2:]

In [26]:
y_train.shape

(281, 2)

In [27]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [28]:
def initialize_model():
   
    model  = models.Sequential()
    
    ### First convolution & max-pooling
    model.add(layers.Conv2D(64, kernel_size=(4, 4), activation='relu', input_shape=(500, 500, 3), padding = "same"))
    model.add(layers.MaxPool2D(pool_size = (2,2)))
    model.add(layers.Dropout(rate=0.2))
    
    ### Second convolution
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding = "same"))
    model.add(layers.Dropout(rate=0.2))


    ### Flattening
    model.add(layers.Flatten())
    
    ### One fully connected
    model.add(layers.Dense(128, activation='relu')) 
    model.add(layers.Dropout(rate=0.5))

    ### Last layer
    model.add(layers.Dense(2, activation='sigmoid')) 

    
    return model

In [29]:
model = initialize_model()
model.summary()

KeyboardInterrupt: 

In [ ]:
def compile_model(model):
    ### Model compilation
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model